In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.functional as F
import torchvision
import tqdm 
import cv2
import os
import glob
import torchvision.transforms as transforms

In [2]:
class Flower():
    def __init__(self):
        self.img_size=(50,50)
        self.file="C:/Users/PARIKALP/AppData/Local/Programs/Python/Scripts/flowers-recognition/flowers"
        self.Daisy="C:/Users/PARIKALP/AppData/Local/Programs/Python/Scripts/flowers-recognition/flowers/daisy/"
        self.Dandelion="C:/Users/PARIKALP/AppData/Local/Programs/Python/Scripts/flowers-recognition/flowers/dandelion/"
        self.Rose=Rose="C:/Users/PARIKALP/AppData/Local/Programs/Python/Scripts/flowers-recognition/flowers/rose/"
        self.Sunflower="C:/Users/PARIKALP/AppData/Local/Programs/Python/Scripts/flowers-recognition/flowers/sunflower/"
        self.Tuplip="C:/Users/PARIKALP/AppData/Local/Programs/Python/Scripts/flowers-recognition/flowers/tulip/"
        self.labels={self.Daisy:0,self.Dandelion:1,self.Rose:2,self.Sunflower:3,self.Tuplip:4}
        self.trainning=[]

    def make_training_data(self):
        for label in self.labels:
            for img in tqdm.tqdm(os.listdir((label))):
                if "jpg" in img:
                    path=os.path.join(label,img)
                    img=cv2.imread(path,cv2.IMREAD_GRAYSCALE)
                    img=cv2.resize(img,(self.img_size))
                    self.trainning.append([np.array(img),np.eye(5)[self.labels[label]]])
                    
        np.random.shuffle(self.trainning)
        np.save("trainning_data.npy", self.trainning)

flower=Flower()
flower.make_training_data()
training_data = np.load("trainning_data.npy", allow_pickle=True)
print(len(training_data))

100%|████████████████████████████████████████████████████████████████████████████████| 984/984 [00:41<00:00, 23.75it/s]


4323


In [3]:
len(training_data)

4323

In [4]:
X=torch.Tensor([i[0] for i in training_data]).view(-1,50,50)
X=X/255.0
y=torch.Tensor([i[1] for i in training_data])

In [56]:
len(X),len(y)

(4323, 4323)

In [5]:
X.shape,y.shape

(torch.Size([4323, 50, 50]), torch.Size([4323, 5]))

In [6]:
VAL_PCT = 0.8 # lets reserve 10% of our data for validation
val_size = int(len(X)*VAL_PCT)
val_size

3458

In [7]:
train_X=X[:-val_size]
train_y=y[:-val_size]
test_X=X[-val_size:]
test_y=y[-val_size:]

In [63]:
len(train_X),len(train_y)

(865, 865)

In [9]:
train_X.shape,train_y.shape,test_X.shape,test_y.shape

(torch.Size([865, 50, 50]),
 torch.Size([865, 5]),
 torch.Size([3458, 50, 50]),
 torch.Size([3458, 5]))

In [20]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.cnn1=nn.Conv2d(in_channels=1,out_channels=16,kernel_size=5)
        self.relu1=nn.ReLU()
        self.max1=nn.MaxPool2d(kernel_size=2)
            
        self.cnn2=nn.Conv2d(in_channels=16,out_channels=32,kernel_size=5)
        self.relu2=nn.ReLU()
        self.max2=nn.MaxPool2d(kernel_size=2)
        
        self.fc1=nn.Linear(32*9*9,5)
        
    def forward(self,x):
        out=self.cnn1(x)
        out=self.relu1(out)
        out=self.max1(out)
        
        out=self.cnn2(out)
        out=self.relu2(out)
        out=self.max2(out)
        
        out=out.view(out.size(0),-1)
        out=self.fc1(out)
        return out
        
        
        
model=CNN()
model

CNN(
  (cnn1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1))
  (relu1): ReLU()
  (max1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (cnn2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
  (relu2): ReLU()
  (max2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=2592, out_features=5, bias=True)
)

In [21]:
criterion=nn.MSELoss()
optimizer=torch.optim.SGD(model.parameters(),lr=0.01)

In [22]:
ATCH_SIZE = 100
EPOCHS = 100
BATCH_SIZE=100
for epoch in range(EPOCHS):
    for i in tqdm.tqdm(range(0, len(train_X), BATCH_SIZE)): # from 0, to the len of x, stepping BATCH_SIZE at a time. [:50] ..for now just to dev
        #print(f"{i}:{i+BATCH_SIZE}")
        batch_X = train_X[i:i+BATCH_SIZE].view(-1, 1, 50, 50)
        batch_y = train_y[i:i+BATCH_SIZE]

        model.zero_grad()

        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()    # Does the update

    print(f"Epoch: {epoch}. Loss: {loss.data}")

100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:09<00:00,  1.06it/s]


Epoch: 0. Loss: 0.1672549545764923


100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:10<00:00,  1.14s/it]


Epoch: 1. Loss: 0.16028527915477753


100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:07<00:00,  1.72it/s]


...

Epoch: 97. Loss: 0.14572933316230774


100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:03<00:00,  2.51it/s]


Epoch: 98. Loss: 0.14563797414302826


100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:03<00:00,  3.15it/s]


Epoch: 99. Loss: 0.14554859697818756
